In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [5]:
class EventSimulator:
    """Simulates event camera data based on statistical mechanics models."""
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height

    def generate_noise_background(self, duration_sec: float, event_rate_hz: float):
        """
        Generates noise events where each pixel acts as an independent Poisson process.
        
        Args:
            duration_sec (float): Length of the simulation in seconds.
            event_rate_hz (float): Average noise rate PER PIXEL in Hz (events/sec).
            
        Returns:
            events (np.ndarray): Structured array of shape (N, 4) -> [x, y, t, p]
                                 sorted by timestamp t.
        """
        # 1. Calculate macroscopic parameters (Grand Canonical Ensemble view)
        total_pixels = self.width * self.height
        total_rate_lambda = total_pixels * event_rate_hz
        
        # Expected number of events in the window
        expected_count = total_rate_lambda * duration_sec
        
        # 2. Draw total event count N from Poisson distribution
        # For large lambda, this approximates a Gaussian, but Poisson is physically exact.
        num_events = np.random.poisson(expected_count)
        
        if num_events == 0:
            return np.zeros((0, 4))
            
        print(f"Generating {num_events} noise events (Target Rate: {event_rate_hz} Hz/pix)")

        # 3. Distribute events uniformly in Space-Time (Entropy maximization)
        # x, y coordinates (integers)
        x = np.random.randint(0, self.width, size=num_events)
        y = np.random.randint(0, self.height, size=num_events)
        
        # t timestamps (floats in seconds, usually microsecond precision in hardware)
        t = np.random.uniform(0, duration_sec, size=num_events)
        
        # p polarity (0 or 1, -1 or +1). Coin flip for random noise.
        p = np.random.randint(0, 2, size=num_events) * 2 - 1  # Result: -1 or 1

        # 4. Sort by timestamp (Crucial: Event streams are always causal/monotonic)
        events = np.column_stack((x, y, t, p))
        events = events[events[:, 2].argsort()]
        
        return events

    @staticmethod
    def plot_spacetime_3d(events, title="Event Cloud (Microstate)"):
        """
        Visualizes the events as a 3D scatter plot (x, y, t).
        """
        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')
        
        # Downsample for plotting if too heavy
        if len(events) > 5000:
            idx = np.random.choice(len(events), 5000, replace=False)
            plot_events = events[idx]
        else:
            plot_events = events

        xs, ys, ts, ps = plot_events.T
        
        # Color by polarity (Red = +1, Blue = -1)
        colors = ['red' if p > 0 else 'blue' for p in ps]
        
        ax.scatter(xs, ts, ys, c=colors, s=2, alpha=0.5)
        
        ax.set_xlabel('X Pixel')
        ax.set_ylabel('Time (s)')
        ax.set_zlabel('Y Pixel')
        ax.set_title(title)
        plt.show()

    def accumulate_frames(self, events: np.ndarray, dt: float, duration_sec: float, mode: str = 'activity'):
        """
        Slices the event stream into discrete frames of duration dt.
        
        Args:
            events: The structured event array [x, y, t, p].
            dt: The integration window duration in seconds (e.g., 0.05 for 50ms).
            duration_sec: Total duration of the simulation.
            mode: 'activity' (absolute count of events) or 'polarity' (sum of polarities).
            
        Returns:
            frames: A 3D numpy array of shape (num_frames, height, width).
        """
        num_frames = int(np.ceil(duration_sec / dt))
        frames = np.zeros((num_frames, self.height, self.width), dtype=int)
        
        if len(events) == 0:
            return frames
            
        # Extract columns
        xs = events[:, 0].astype(int)
        ys = events[:, 1].astype(int)
        ts = events[:, 2]
        ps = events[:, 3]
        
        # Calculate which frame index each event belongs to
        frame_indices = (ts // dt).astype(int)
        
        # Filter out events that might fall outside the exact duration
        valid_mask = frame_indices < num_frames
        xs, ys, frame_indices, ps = xs[valid_mask], ys[valid_mask], frame_indices[valid_mask], ps[valid_mask]
        
        # Determine the values to add to the pixels
        if mode == 'activity':
            vals = np.ones_like(ps) # Just count the presence of an event
        elif mode == 'polarity':
            vals = ps # Add +1 for positive polarity, -1 for negative
            
        # Efficiently accumulate events into the 3D frame tensor
        # np.add.at is perfect for this "histogram-like" binning operation
        np.add.at(frames, (frame_indices, ys, xs), vals)
        
        return frames

    def animate_frames(self, frames, dt: float, filename: str = None):
        """
        Creates a video animation of the accumulated frames, optimized for Jupyter Notebooks.
        
        Args:
            frames: 3D numpy array from accumulate_frames.
            dt: The integration window (used to calculate playback FPS).
            filename: If provided, saves the animation as an mp4.
            
        Returns:
            HTML object displaying the video in a Jupyter Notebook.
        """
        fig, ax = plt.subplots(figsize=(6, 6))
        
        # Determine appropriate color limits based on the data
        vmax = np.max(np.abs(frames))
        vmin = 0 if np.min(frames) >= 0 else -vmax
        cmap = 'gray' if vmin == 0 else 'coolwarm' # coolwarm is good for polarity (-1 to 1)
        
        # Initialize the image plot
        img = ax.imshow(frames[0], cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest')
        ax.set_title(f"Time: 0.000 s\nFrame: 0")
        ax.axis('off')
        
        # Update function for FuncAnimation
        def update(frame_idx):
            img.set_data(frames[frame_idx])
            current_time = frame_idx * dt
            ax.set_title(f"Time: {current_time:.3f} s\nFrame: {frame_idx}")
            return [img]
            
        # Calculate FPS based on physics integration time
        fps = int(1.0 / dt)
        interval_ms = dt * 1000
        
        anim = animation.FuncAnimation(
            fig, update, frames=len(frames), 
            interval=interval_ms, blit=True
        )
        
        plt.close(fig) # Prevent static plot from showing below the animation
        
        if filename:
            # Requires ffmpeg installed on your system
            anim.save(filename, writer='ffmpeg', fps=fps)
            print(f"Saved video to {filename}")
            
        # This renders perfectly in Jupyter Notebooks
        return HTML(anim.to_jshtml())


In [ ]:
# 1. Initialize and generate noise (from the previous step)
sim = EventSimulator(width=128, height=128)
duration = 10.0 # 1 second
dt = 0.05 # 50 ms integration window (20 FPS)

noise_events = sim.generate_noise_background(duration_sec=duration, event_rate_hz=10.0)

# 2. Accumulate into frames
# Using 'activity' mode maps well to the Poisson intensity models discussed.
frames = sim.accumulate_frames(noise_events, dt=dt, duration_sec=duration, mode='polarity')

# 3. Animate (This will display a video player directly in your Jupyter cell)
# The animation will look like "TV static", representing the disordered phase.
sim.animate_frames(frames, dt=dt)

Generating 164036 noise events (Target Rate: 10.0 Hz/pix)
